In [ ]:
# Install required packages in Google Colab
!pip install ctgan gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

In [ ]:
import pandas as pd
import tempfile
import math
from ctgan import CTGAN
import gradio as gr

In [ ]:
def generate_synthetic_data(file, epochs, num_samples):
    # Load uploaded CSV
    df = pd.read_csv(file.name)

    # Automatically detect discrete (categorical) columns
    discrete_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

    # Define chunking strategy
    chunk_size = 1000
    num_chunks = math.ceil(len(df) / chunk_size)
    generated_data = []

    for i in range(num_chunks):
        chunk = df.iloc[i*chunk_size : (i+1)*chunk_size].reset_index(drop=True)

        # Skip very small chunks
        if len(chunk) < 10:
            continue

        # Recompute discrete columns for this chunk
        chunk_discrete = chunk.select_dtypes(include=['object', 'category']).columns.tolist()

        # Train CTGAN on the chunk
        ctgan = CTGAN()
        ctgan.fit(chunk, chunk_discrete, epochs=int(epochs))

        # Sample synthetic data
        samples = ctgan.sample(int(num_samples / num_chunks))
        generated_data.append(samples)

    # Combine all generated samples
    final_df = pd.concat(generated_data, ignore_index=True)

    # Save to temporary CSV
    temp_path = tempfile.NamedTemporaryFile(delete=False, suffix=".csv").name
    final_df.to_csv(temp_path, index=False)

    return temp_path

iface = gr.Interface(
    fn=generate_synthetic_data,
    inputs=[
        gr.File(label="Upload CSV"),
        gr.Number(label="Epochs", value=5),
        gr.Number(label="Number of Samples", value=100)
    ],
    outputs=gr.File(label="Download Synthetic CSV"),
    title="CTGAN Synthesizer",
    description="Upload a dataset, train CTGAN in memory-safe batches, and download synthetic data."
)

# iface.launch(share=True)
iface.launch(share=True, show_error=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7014e1db5ab8050ab7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
